In [43]:
###########################################################
# FUNCIONES PARA REDUCIR POR EQUIVALENCIA MONOMIAL
###########################################################
from itertools import permutations

def column_stack(cols):
    """
    Reconstruye una matriz a partir de una lista de vectores columna.
    """
    r = cols[0].length()
    c = len(cols)
    R = cols[0].base_ring()
    M = matrix(R, r, c)
    for j in range(c):
        for i in range(r):
            M[i, j] = cols[j][i]
    return M

from itertools import permutations

def forma_canonica_monomial(G):
    """
    Forma canónica completa bajo equivalencia monomial:
      - prueba todas las permutaciones de columnas n!
      - para cada permutación: lleva a echelon_form, crea copia mutable (imporante, no anda sino)
        normaliza cada columna para que el primer coeficiente no nulo sea 1,
        y toma la tupla row-major como clave lexicográfica mínima.
    Devuelve la matriz canónica (mutable) correspondiente a la permutación mínima.
    """
    k = G.nrows()
    n = G.ncols()
    mejor = None  # (clave_tuple, matriz_canónica)
    BR = G.base_ring()

    for p in permutations(range(n)):
        # construir matriz con columnas en el orden p
        cols = [G.column(j) for j in p]
        entries = [cols[j][i] for i in range(k) for j in range(n)]  # row-major
        Gp = matrix(BR, k, n, entries).echelon_form()

        # crear copia mutable de Gp (uso list(Gp.list()) para forzar nueva matriz)
        Gnorm = matrix(BR, k, n, list(Gp.list()))

        # normalizar cada columna: primer coef != 0 -> hacerlo 1
        for j in range(n):
            # obtener columna como lista
            col = [Gnorm[i, j] for i in range(k)]
            pivot = None
            for x in col:
                if x != 0:
                    pivot = x
                    break
            if pivot is not None:
                inv = pivot**(-1)
                for i in range(k):
                    Gnorm[i, j] = inv * Gnorm[i, j]

        clave = tuple(Gnorm.list())  # representación inmutable row-major
        if (mejor is None) or (clave < mejor[0]):
            # guardamos una copia (por seguridad)
            mejor = (clave, matrix(BR, k, n, list(Gnorm.list())))

    # devolver la matriz canónica (segunda componente)
    return mejor[1] if mejor is not None else matrix(G.base_ring(), k, n, list(G.list()))


def filtrar_no_equivalentes_completo(lista):
    """
    Filtra la lista de matrices generadoras guardando un representante original
    por cada clase monomial. Usa forma_canonica_monomial para comparar clases.
    """
    rep_map = {}
    for G in lista:
        CF = forma_canonica_monomial(G)
        key = tuple(CF.list())
        if key not in rep_map:
            rep_map[key] = G    # guardamos la matriz ORIGINAL como representante
    return list(rep_map.values())

###########################################################
# FUNCIONES PARA REDUCIR POR EQUIVALENCIA MONOMIAL
###########################################################


def construir_sigma_para_B(B, K, x):
    """
    Construye el automorfismo sigma asociado a B sobre K.<x>.
    Devuelve el objeto sigma (o None en caso de error).
    """
    a00 = B[0,0]; a01 = B[0,1]; a10 = B[1,0]; a11 = B[1,1]
    try:
        A00 = K(a00); A01 = K(a01); A10 = K(a10); A11 = K(a11)
    except Exception:
        A00 = a00; A01 = a01; A10 = a10; A11 = a11
    racional = (A00*x + A01) / (A10*x + A11)
    try:
        sigma = K.hom((racional,))   # tupla de 1 elemento (K tiene 1 generador)
    except Exception:
        return None
    return sigma

def lugares_fijos_por_sigma(sigma, monicirred2, R):
    """
    Dada sigma y la lista de polinomios irreducibles monicos de grado 2,
    devuelve la lista de aquellos polinomios Q tales que sigma(Q) fija el lugar.
    IMPORTANTE: se pasa R (PolynomialRing) para coerciones seguras.
    """
    lg2 = []
    for pol in monicirred2:
        try:
            sigmapol = sigma(pol)
        except Exception:
            continue
        # coercionar numeradores/denominadores al mismo anillo R
        try:
            arriba = R(sigmapol.numerator())
            abajo  = R(sigmapol.denominator())
            numpol = R(pol.numerator())
        except Exception:
            # si la coerción falla, saltamos ese pol
            continue
        testeo = [numpol.divides(arriba), numpol.divides(abajo)]
        if testeo == [True, False]:
            lg2.append(pol)
    return lg2

def construir_orbitas_para_B(B, k, a, q, elem, menoselem, K, n_expected):
    """
    Construye las orbitas finitas asociadas a B
    Filtra y devuelve sólo las órbitas de longitud exactamente n_expected.
    """
    places_finite = K.places_finite()
    Binv = Matrix(k,2,[B[1,1], -B[0,1], -B[1,0], B[0,0]])
    orbitas = []
    for i in range(q):
        j = None
        for hache in range(q):
            if menoselem[i] == elem[hache]:
                j = hache
                break
        if j is None:
            continue
        if any(places_finite[j] in orb for orb in orbitas):
            continue
        orbita = [places_finite[j]]
        val = a**i
        brk2 = False
        while (not brk2) and len(orbita) < n_expected:
            num = (Binv[0,0]*val + Binv[0,1])
            deno = (Binv[1,0]*val + Binv[1,1])
            if deno != 0:
                new = num/deno
                indice = None
                for idx in range(q):
                    if new == menoselem[idx]:
                        indice = idx
                        break
                if indice is None:
                    orbita.append(K.places()[0])
                    brk2 = True
                else:
                    orbita.append(places_finite[indice])
                    val = new
            else:
                orbita.append(K.places()[0])
                brk2 = True
        if len(orbita) == n_expected:
            orbitas.append(orbita)
    return orbitas

def generar_matrices_generadoras_para_lg2(orbitas, lg2fijos, O, k):
    """
    Para cada pol Q en lg2fijos y para cada orbita construye la matriz generadora
    """
    MATGEN_local = []
    for Q in lg2fijos:
        try:
            I = O.ideal(Q)
            G = I.divisor_of_zeros()
            Base = G.basis_function_space()
        except Exception:
            Base = []
        if not Base:
            continue
        for orb in orbitas:
            lista = []
            for base in Base:
                for P in orb:
                    try:
                        lista.append(base.evaluate(P))
                    except Exception:
                        lista.append(k(0))
            filas = len(Base); columnas = len(orb)
            if len(lista) < filas*columnas:
                lista += [k(0)]*(filas*columnas - len(lista))
            try:
                M = Matrix(k, filas, columnas, lista)
                cod = LinearCode(M)
                gen = cod.systematic_generator_matrix()
                MATGEN_local.append(gen)
            except Exception:
                continue
    return MATGEN_local

In [44]:
muestra = [9]
for q in muestra:
    print(91*'-')
    print('INICIO DEL CASO Fq con q =', q)
    k.<a> = GF(q)                              # Generamos el cuerpo
    ma = a.charpoly('y')                       # Generamos el polinomio caracteristico
    print(31*'-')
    print('El polinomio caracteristico es', ma)
    
    elem = []                                  # elementos explicitos del cuerpo
    if is_prime(q):                            # si el orden del cuerpo es primo
        for i in k:                            # metemos todos los elementos
            elem.append(i)                                 
    else:                                      # Si el orden no es primo
        for i in range(q):                     # los metemos en funcion del caracteristico
            if i == 0:                         # Esto nos servira mucho mas adelante
                elem.append(a*0)               # cuando trabajemos con las matrices de los codigos
            else:
                elem.append(a**i)
    menoselem = [ -e for e in elem ]  # Generamos el conjunto de menos los elementos del cuerpo
    
    R.<t> = PolynomialRing(k)         # anillo de funciones
    monicirred2 = []                  # conjunto de los polinomios monicos irreducibles de grado 2
    for pol in R.polynomials(2):
        if pol.is_irreducible() and list(pol)[2] == 1:
            monicirred2.append(pol)
    print(31*'-')
    print('Hay', len(monicirred2), 'polinomios monicos irreducibles de grado 2')
    
    K.<x> = FunctionField(k)           # cuerpoo de funciones
    O = K.maximal_order()              # orden del cuerpo de funciones

    # construir IMS (matrices asociadas a cada irreducible)
    IMS = []
    for pmi in monicirred2:
        pmilista = list(pmi)
        a0 = pmilista[0]
        a1 = pmilista[1]
        for tval in k:
            A = [tval, -a0, 1, a1 + tval]
            B = Matrix(k,2,A)
            if B.det() != 0:
                IMS.append(B)

    # calculo de ordenes de cada matriz (matord)
    matord = []
    for matriz in IMS:
        n = 1
        while True:
            if matriz**n == (matriz**n)[0,0]*identity_matrix(k,2):
                matord.append(n)
                break
            n += 1

    NN = Set(matord) #Conjunto de matrices de orden n
    N = []
    for n in NN:
        if (n < q+1) and (n > 3) and (n.divides(q+1)):
            N.append(n)
    print(31*'-')
    print('Las longitudes posibles para codigos sigma ciclicos son', N)

    # Para cada matriz de orden n, buscamos el codigo ag sigma ciclico 
    for n in N:
        print(31*'-')
        print('ESTUDIAREMOS AHORA CODIGOS DE LONGITUD', n, 'Y DIVISORES G DE GRADO 2')

        # matn = matrices de orden n
        matn = [ IMS[i] for i in range(len(matord)) if matord[i] == n ]
        print(31*'-')
        print('Hay', len(matn), 'matrices de orden', n, 'sin reducir PGL2')

        MATGEN = []
        CONTEO = []

        # procesar cada B en matn
        for B in matn:
            sigma = construir_sigma_para_B(B, K, x)
            if sigma is None:
                # si no pudimos construir sigma saltamos B
                CONTEO.append(0)
                continue

            # lugares fijos de grado 2
            lg2fijos = lugares_fijos_por_sigma(sigma, monicirred2,R)
            cantlg = len(lg2fijos)
            CONTEO.append(cantlg)

            # construir orbitas de longitud n
            orbitas = construir_orbitas_para_B(B, k, a, q, elem, menoselem, K, n)

            # generar matrices generadoras para esos lg2fijos y orbitas
            mats_loc = generar_matrices_generadoras_para_lg2(orbitas, lg2fijos, O, k)
            MATGEN.extend(mats_loc)

        Theta = list(Set(MATGEN))
        theta = len(Theta)
        print(31*'-')
        print('HAY', theta, 'MATRICES GENERADORAS DE CODIGOS DISTINTOS, PERO QUIZAS EQUIVALENTES:')

        if theta == 0:
            print("Theta está vacío: no hay matrices generadoras para analizar.")
            continue

        # Filtrado rápido por equivalencia monomial (obtenemos representantes originales)
        representantes = filtrar_no_equivalentes_completo(Theta)
        print(f"Existen {len(representantes)} códigos no equivalentes (clases monomiales):\n")

        for i, G in enumerate(representantes, start=1):
            print(f"--- Representante {i} ---")
            # imprimo la MATRIZ ORIGINAL en forma sistemática para ver su forma
            try:
                print(G.echelon_form())
            except Exception:
                print(G)
            print()

        print('Debemos chequear si son o no monomialmente equivalentes entre sí (equiv. por escalamientos columna).')

-------------------------------------------------------------------------------------------
INICIO DEL CASO Fq con q = 9
-------------------------------
El polinomio caracteristico es y^2 + 2*y + 2
-------------------------------
Hay 36 polinomios monicos irreducibles de grado 2
-------------------------------
Las longitudes posibles para codigos sigma ciclicos son [5]
-------------------------------
ESTUDIAREMOS AHORA CODIGOS DE LONGITUD 5 Y DIVISORES G DE GRADO 2
-------------------------------
Hay 144 matrices de orden 5 sin reducir PGL2
-------------------------------
HAY 20 MATRICES GENERADORAS DE CODIGOS DISTINTOS, PERO QUIZAS EQUIVALENTES:
Existen 6 códigos no equivalentes (clases monomiales):

--- Representante 1 ---
[      1       0       0       2     2*a]
[      0       1       0   a + 1 2*a + 2]
[      0       0       1 2*a + 1 2*a + 2]

--- Representante 2 ---
[  1   0   0 2*a   1]
[  0   1   0   a   0]
[  0   0   1   1   0]

--- Representante 3 ---
[      1       0       

In [45]:
from itertools import permutations, combinations, product

def find_monomial_equivalence(G1, G2):
    """
    Busca U (kxk invertible), D (vector de escalares no nulos de longitud n),
    P (perm de columnas) tales que G2 = U * G1 * diag(D) * P.
    Devuelve (True, (U, D_vec, perm)) si encuentra una solución,
    o (False, None) si no existe.
    """
    k = G1.nrows()
    n = G1.ncols()
    F = G1.base_ring()
    nonzero = [e for e in F if e != 0]

    # Sanity checks
    if (G2.nrows() != k) or (G2.ncols() != n):
        return False, None

    # Precompute G1 columns list for quick access
    cols1 = [G1.column(j) for j in range(n)]
    cols2 = [G2.column(j) for j in range(n)]

    # Para cada permutación de columnas p (tupla de índices)
    for p in permutations(range(n)):
        # Construimos G1 permutada (columns permuted)
        cols1p = [cols1[j] for j in p]  # list of vectors
        # Escoger combinaciones de k columnas que formen base (evitar intentos inútiles)
        for basis_idx in combinations(range(n), k):
            # Indices into permuted columns
            basis_cols = [cols1p[i] for i in basis_idx]
            # Form matrix S (k x k) con esas columnas
            S = matrix(F, k, k, [basis_cols[j][i] for i in range(k) for j in range(k)])
            if S.det() == 0:
                continue  # not a basis

            # T = columnas correspondientes de G2 (usamos mismas posiciones en p)
            T = matrix(F, k, k, [cols2[i][r] for r in range(k) for i in basis_idx])

            # Ahora probamos posibles valores para los k escalares en la base
            # (d for columns in basis)
            for d_basis in product(nonzero, repeat=k):
                # Construir S' = S * diag(d_basis) (escalariza cada columna)
                S_prime = matrix(F, k, k, [basis_cols[j][i]*d_basis[j] for i in range(k) for j in range(k)])
                if S_prime.det() == 0:
                    continue
                # Podemos resolver U = T * S_prime^{-1}
                try:
                    U = T * S_prime.inverse()
                except Exception:
                    continue

                # Ahora para cada columna j (in permuted order) resolvemos d_j tal que
                # U * (cols1p[j] * d_j) == cols2[j]  -> d_j debe satisfacer proportionalidad
                D_vec = [None]*n
                ok = True
                for j in range(n):
                    v1 = cols1p[j]
                    v2 = cols2[j]
                    Uv1 = U * v1  # vector
                    # Si v2 is zero vector
                    if all(x == 0 for x in v2):
                        # then Uv1 must be zero for some d_j (but Uv1*d=0 => either Uv1=0 or d=0 impossible)
                        if all(x == 0 for x in Uv1):
                            D_vec[j] = nonzero[0]  # cualquier no nulo; usar 1
                            continue
                        else:
                            ok = False
                            break
                    # Buscamos d such that d * Uv1 == v2  => for coordinates where Uv1 !=0, d = v2_i / Uv1_i consistent
                    d_val = None
                    for idx in range(k):
                        if Uv1[idx] != 0:
                            cand = v2[idx] * (Uv1[idx]**(-1))
                            d_val = cand
                            break
                    if d_val is None:
                        # Uv1 all zero but v2 not zero -> imposible
                        ok = False
                        break
                    # comprobar que para todas coordenadas se cumple v2 == d_val * Uv1
                    for idx in range(k):
                        if v2[idx] != d_val * Uv1[idx]:
                            ok = False
                            break
                    if not ok:
                        break
                    # debemos asegurarnos d_val != 0
                    if d_val == 0:
                        ok = False
                        break
                    D_vec[j] = d_val
                if not ok:
                    continue

                # Ahora tenemos D_vec wrt permuted columns order; construir diagonal matrix D_full
                # But recall cols1p correspond to permuted order p; we want D in original column order.
                # D_vec is in permuted order; convert back to original order: D_original[col_index] = D_vec[pos in p]
                D_original = [None]*n
                for pos, col_index in enumerate(p):
                    D_original[col_index] = D_vec[pos]

                # Verificamos finalmente: U * (G1 * diag(D_original) * P_matrix) == G2
                # Build diag(D_original) matrix
                Dmat = diagonal_matrix(F, D_original)
                # Build permutation matrix Pmat that reorders columns: G1 * Dmat * Pmat should match columns order of G1p * D_vec
                # Simpler: compute G1 * Dmat, then permute columns to order p
                G1D = G1 * Dmat
                # permute columns of G1D according to p to get G1D_permuted
                cols_G1D = [G1D.column(j) for j in range(n)]
                cols_G1D_permuted = [cols_G1D[j] for j in p]
                G1Dperm = matrix(F, k, n, [cols_G1D_permuted[j][i] for i in range(k) for j in range(n)])
                # finally compute U * G1Dperm
                if U * G1Dperm == G2:
                    # Success: return U, D_original, p (permutation tuple)
                    return True, (U, D_original, p)
                # else continue searching
    return False, None


# Helper to test list of candidates against two ground-truth matrices:
def check_candidates_against_two(representantes, target1, target2):
    """
    Para cada G en representantes intenta ver si es monomialmente equivalente a target1 o target2.
    Imprime el resultado y (si existe) la transformación concreta U,D,P.
    """
    for idx, G in enumerate(representantes, start=1):
        print("\n--- Chequeando representante", idx, "---")
        print(G)
        ok1, sol1 = find_monomial_equivalence(G, target1)
        if ok1:
            U, D, p = sol1
            print("Equivalente a TARGET1. Transformación encontrada: P =", p, "D =", D)
            continue
        ok2, sol2 = find_monomial_equivalence(G, target2)
        if ok2:
            U, D, p = sol2
            print("Equivalente a TARGET2. Transformación encontrada: P =", p, "D =", D)
            continue
        print("No equivalente a TARGET1 ni TARGET2.")


In [46]:
# pon aquí las dos matrices que deben ser los representantes teóricos (ejemplo ya desde tu salida)
T1 = matrix(k, 3, 5, [1,0,0,2,2*a, 0,1,0,a+1,2*a+2, 0,0,1,2*a+1,2*a+2])  # fila-major
T2 = matrix(k, 3, 5, [1,0,0,2*a,1, 0,1,0,a,0, 0,0,1,1,0])

# tu lista de representantes (ej. la que obtuviste)
rep_list = representantes  # si ya la tienes

check_candidates_against_two(rep_list, T1, T2)



--- Chequeando representante 1 ---
[      1       0       0       2     2*a]
[      0       1       0   a + 1 2*a + 2]
[      0       0       1 2*a + 1 2*a + 2]
Equivalente a TARGET1. Transformación encontrada: P = (0, 1, 2, 3, 4) D = [a, a, a, a, a]

--- Chequeando representante 2 ---
[  1   0   0 2*a   1]
[  0   1   0   a   0]
[  0   0   1   1   0]
Equivalente a TARGET2. Transformación encontrada: P = (0, 1, 2, 3, 4) D = [a, a, a, a, a]

--- Chequeando representante 3 ---
[      1       0       0       a   a + 1]
[      0       1       0       2 2*a + 1]
[      0       0       1 2*a + 2       2]
Equivalente a TARGET1. Transformación encontrada: P = (0, 2, 1, 4, 3) D = [a, 2*a + 1, 2*a, 2*a, 2*a + 1]

--- Chequeando representante 4 ---
[      1       0       0 2*a + 2   a + 2]
[      0       1       0 2*a + 2       a]
[      0       0       1     2*a   a + 2]
No equivalente a TARGET1 ni TARGET2.

--- Chequeando representante 5 ---
[  1   1   0   0   1]
[  0   0   1   0 2*a]
[  0   0 

In [49]:
ok46, sol46 = find_monomial_equivalence(representantes[3], representantes[5])  # 3→rep4, 5→rep6

if ok46:
    print("Rep4 y Rep6 SON equivalentes.")
    U, D, p = sol46
    print("Transformación encontrada:")
    print("Permutación:", p)
    print("Escalas:", D)
else:
    print("Rep4 y Rep6 NO son equivalentes -> forman clases distintas.")


Rep4 y Rep6 NO son equivalentes -> forman clases distintas.
